# Intelligent Plant App Store and Data Core Client (Python)

## Setup
- Register as a developer with the intelligent plant app store.
- Create a new app by clicking "+" under Developer > Applications
- Add the URL of the current page as an authorized redirect URL
- Replace the app ID below with the ID of your app


This sheet was testsed on "IP Datasource 2"

In [ ]:
app_id = "73b72bcdd6234977a7f692c7972567a4"

Import libraries we're going to use

In [ ]:
import math
import urllib.parse
from functools import reduce

import IPython.display as display
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Import the Intelligent Plant App Store and Data Core clients.

In [ ]:
import intelligent_plant.app_store_client as app_store_client
import intelligent_plant.utility as utility

Import the modules necessary for generating the log in URL

Get the URL if this notebook

In [ ]:
current_url = None

In [ ]:
%%javascript
IPython.notebook.kernel.execute("current_url = '" + window.location + "'");

Check the URL to see if it has the access token.

If it doesn't: generate a login link

In [ ]:
parsed = urllib.parse.urlparse(current_url)
token = None

try:
    token = urllib.parse.parse_qs(parsed.fragment)['access_token'][0]
except:
    print("Token is not valid")
    print(token)

if (not token):
    url = app_store_client.get_implicit_grant_flow_url(app_id, current_url, ["UserInfo", "DataRead"])
    print("Click this link to log in")
    print(url)

In [ ]:
app_store = app_store_client.AppStoreClient(token, None)

In [ ]:
data_core = app_store.get_data_core_client()

## App Store
Instatiate the appstore client

These example queries are based on the functions in examples/example_queries.py

Fetch the current user data

In [ ]:
app_store.get_user_info()

Fetch the current user's balance

In [ ]:
app_store.get_user_balance()

## Data Core

These example queries are based on the functions in examples/example_queries.py

Fetch data sources matching the filter '*' (so all of them)

In [ ]:
data_sources = data_core.get_data_sources(["*"])
dsns = list(map(lambda x: x["Name"]["DisplayName"], data_sources))
dsns

IP Datasource 2 is used for the remainder of these examples

Please ensure that it is the IP Datasource 2 is authroized (i.e. in the above list)

In [ ]:
if (not any(list(map(lambda x: x == 'IP Datasource 2', dsns)))):
    print ("Please authorize IP Datasource 2 for Python API Test https://appstore.intelligentplant.com/Security/Apps")

In [ ]:
dsn = "IP Datasource 2"

Fetch all the tags in the "IP Datasource 2" data source. TPage size is 113 because that is the number of tags in the data source.

In [ ]:
tags = data_core.get_tags(dsn, 1, 113)
tags

We can use the first results as examples for the rest of this note book.

In [ ]:
tag = tags[0]["Id"]

Take the first available tag and do a historical query over the last 30 days

The parameters are:
- Fully Qualified data source name
- Tag  (can be a list)
- Data function, "plot", "interp", "max", "min", "avg" and "raw" (raw is not recommended due to high volumes of data)
- The query start time (can be relative e.g. "-30d" or absolute "01/01/2019"
- The query end time
- The sample interval

Optionally takes annotations=True to include annotations

We can request the plot dat for the last 30 days of this tag like this. There will be one data point for each day (so 30 points). The "*" means now

In [ ]:
#request some data (from the first tag the tag search returned)
data = data_core.get_plot_data({dsn: [tag]}, "*-30d", "*", 30)
data

There is a built in query function to convert the query result into a data frame

In [ ]:
data_frame = utility.query_result_to_data_frame(data)

In [ ]:
data_frame

We can plot this data to see what it looks like

In [ ]:
data_frame.plot()

We can get the data for multiple tags simulataneously like so

In [ ]:
#fetch the data for all the tags we found
tag_names =  list(map(lambda x: x["Id"], tags))
#fetch the data for all the tags we found
    
many_data = data_core.get_plot_data({dsn: tag_names}, "*-30d", "*", 30)

many_data_frame = utility.query_result_to_data_frame(many_data)

In [ ]:
many_data_frame

We can plot it the same way as before

In [ ]:
#plot the data frame
many_data_frame.plot(title="Lots of Data", legend=False)

You can also use the "now" function to query the current value of a tag.

In [ ]:
data_core.get_snapshot_data({dsn: [tag]})

### Data Processing

This query will get all of the data in the demo "IP Datasource 2"

In [ ]:
all_data = data_core.get_raw_data({dsn: tag_names}, "2014-03-03", "2014-03-13", 239)

all_data_frame = utility.query_result_to_data_frame(all_data)

We can use pandas functions to normalise the data between 0 and 1. This is useful for lot's of data processing applications

In [ ]:
normalized_df=(all_data_frame-all_data_frame.min())/(all_data_frame.max()-all_data_frame.min())
normalized_df

In [ ]:
#plot the data frame
normalized_df.plot(title="Normalized", legend=False)

Now that the data is normalised we can use it as the training and testing data for a multi layer perceptron.

In [ ]:
from sklearn.neural_network import MLPRegressor

Seperate the data into seperate trainigna and testing parts

In [ ]:
training_input = normalized_df.iloc[:-10,:-1]
training_output = normalized_df.iloc[:-10,-1]

testing_input = normalized_df.iloc[-10:,:-1]
testing_output = normalized_df.iloc[-10:,-1]

We can define the model and train it on our data

In [ ]:
clf = MLPRegressor(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)

In [ ]:
clf.fit(training_input, training_output)

We can then use the model to make predications about the unseen testing data

In [ ]:
prediction = clf.predict(testing_input)

Create a data frame to compare the models prediction to the actual output from the testing data

In [ ]:
prediction_df = pd.DataFrame({ "actual": testing_output, "prediction": prediction})
prediction_df

In [ ]:
prediction_df.plot()